In [24]:
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import datetime


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
#scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import xlrd
from sklearn.preprocessing import Imputer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFECV
import seaborn as sns
import xgboost as xgb
import zipcode
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

sns.set_style("whitegrid")
%matplotlib inline

scipy: 0.19.1
numpy: 1.12.1
matplotlib: 2.0.2
pandas: 0.20.3
sklearn: 0.19.0


In [115]:
pd.set_option("display.max_columns",500)

In [116]:
data = pd.read_csv('SuperZooData/Registration_Data/SuperZoo_2016_Registrants_No_Minors.csv', header = 2)

In [117]:
test_data = pd.read_csv('SuperZooData/Registration_Data/SuperZoo_2017_Registrants_No_Minors.csv', header = 2)

In [ ]:
plotting = data['AddDate Date'].value_counts().sort_values(ascending=True)

In [ ]:
plotting.index.sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(150, 150))
plt.plot(plotting.index.sort_values(ascending=True), plotting)
plt.xticks(rotation='vertical')

In [ ]:
df = df.sort_values('date', ascending=True)
plt.plot(df['date'], df['count'])
plt.xticks(rotation='vertical')

In [118]:
def dups_with_dif_index_check(data):
    dupes = {}                        # checks to see if duplicated columns with different index values
    for col_a in data.columns:
        dupes[col_a] = []

        for col_b in data.columns:
            if col_b in dupes.keys():
                continue
            if (data.loc[:,col_a] == data.loc[:,col_b]).sum() == len(data.loc[:,col_a]):
                dupes[col_a].append(col_b)    
   
    bad_cols = []   
    for col, dup_cols in dupes.iteritems():
        bad_cols += dup_cols
    bad_cols = set(bad_cols)

    data = data.loc[:,~data.columns.isin(bad_cols)]
    return data

In [119]:
def clear_dups_na(data):
    data = data.loc[:,~data.columns.duplicated()] 
    data = data.dropna(axis=1, how='all')
    data = data.dropna(axis=0, how='all')
    data = dups_with_dif_index_check(data)
    return data

In [122]:
def def_time_modify(data, date_checker):
    data["AddDate Date"] = pd.to_datetime(data["AddDate Date"])
    data['ModifyDate Date'] = pd.to_datetime(data['ModifyDate Date'])
    for a in range(0,len(data.iloc[:,data.columns.get_loc('ModifyDate Date')])):
        if data.iloc[a,data.columns.get_loc('ModifyDate Date')] > date_checker:
            data.iloc[a,data.columns.get_loc('ModifyDate Date')] = data.iloc[a,data.columns.get_loc("AddDate Date")]
    return data

In [123]:
def date_time_update(data, date_checker):
    data = data[data["AddDate Date"] <= date_checker]
    return data
    

In [124]:
def data_drop_other_text(data):
    col = []
    for a in data.columns:
        if "_text" in a or "_other" in a:
            col.append(a)
    data=data.drop(labels = col, axis =1)
    return data

In [125]:
def get_rid_of_nonnumerical_answer(data):
    col = [] # THIS NEEDS TO BE CHANGED FOR NEW DATA SET
    checker = 1
    for a in data.columns:
        if '_codes' in a and checker ==1:
            checker = 0
        if '_codes' not in a and checker == 0:
            col.append(a)
    data = data.drop(col, axis = 1)
    return data

In [126]:
def answers_dummies(data):
    column =[]
    for a in data.columns:
        if "_codes" in a:
            column.append(a)  
    for a in column:
        new_data = data[a].str.get_dummies(sep=',')
        data = pd.concat([data, new_data], axis=1)
        data = data.drop([a], axis = 1)
    return data
    

In [127]:
def data_change_string_drop_dups(data):
    for a in data.columns:
        if data[a].dtype== np.dtype('O'):
            data[a] = data[a].astype('S32')
    cols = list(data)    # THIS DROPS ALL COLUMNS THAT HAVE THE SAME VALUE
    nunique = data.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    data = data.drop(cols_to_drop, axis=1)
    return data

In [128]:
def purchase_dummies(data):
    new_data = data['PurchaseItems'].str.get_dummies(sep=',')
    data = pd.concat([data, new_data], axis=1)
    data = data.drop(['PurchaseItems'], axis = 1)
    return data 

In [129]:
def state_dummies(data):
    s = data.State.value_counts()
    s = data.State.value_counts()[s > (8)].index #8
    holder = pd.get_dummies(data.State)
    holder_smaller = holder[s]
    data = pd.concat([data, holder_smaller], axis=1)
    return data

In [130]:
def check_zip(data):
    data.loc[data.loc[:,'ZipPostal'].str.contains('-', na = False),'ZipPostal'] = data.loc[data.loc[:,'ZipPostal'].str.contains('-', na = False),'ZipPostal'].str.split('-',n = 1, expand = True)[0]
    return data

In [131]:
def zip_distance(data):
    data = check_zip(data)
    geolocator = Nominatim()
    vegas = zipcode.isequal('89178')
    veg = (vegas.lat, vegas.lon)
    col= []
    for idx, val in enumerate (data.ZipPostal):
        try:
            city = zipcode.isequal(val)
            cit = (city.lat, city.lon)
            col.append(vincenty(veg, cit).miles)
        except:
            try:
                city = geolocator.geocode(data.loc[idx,'Country']+" "+val)
                cit = (city.latitude, city.longitude)
                col.append(vincenty(veg, cit).miles)
            except:
                col.append('nan')
    data.insert(0, 'Distance', col, allow_duplicates=False)
   
    return data

In [132]:
def email_split_username(data):
    data.loc[:,'Email_Username'] = data.Email.str.split('@',n = 1, expand = True)[0]
    return data
    

In [133]:
def email_contains_username(data):
    data.loc[:,'Email_Username']=~data.loc[:,'Email_Username'].str.contains('spa|wash|claw|pet|dog|woof|wag|tail|bark|groom|paw|animal|chow|poodle|fur|hound|canine|pup|pooch', regex = True) 
    return data

In [134]:
def email_split_host(data):
    data.loc[:,'Email_Host'] = data.Email.str.split('@',n = 1, expand = True)[1]
    return data

In [135]:
def email_cleanup(data):
    data.Email = data.Email.str.lower()
    data.loc[:,'Email'][~data.loc[:,'Email'].str.contains('@')] = 'a@a'
    return data

In [136]:
def email_contains_hostname(data):
    data.loc[data.loc[:,'Email_Host'].str.contains('spa|wash|claw|pet|dog|woof|wag|tail|bark|groom|paw|animal|chow|poodle|fur|hound|canine|pup|pooch', na = False,regex = True), 'Email_Host'] = 'PET'
    return data

In [137]:
def email_change(data):
    data = email_cleanup(data)
    data = email_split_username(data)
    data = email_split_host(data)
    data = email_contains_username(data)
    data = email_contains_hostname(data)
    return data

In [138]:
def email_dummies(data):
    data = email_change(data)
    s = data.Email_Host.value_counts()
    s = data.Email_Host.value_counts()[s > (48)].index  #48
    holder = pd.get_dummies(data.Email_Host)
    holder_smaller = holder[s]
    data = pd.concat([data, holder_smaller], axis=1)
    return data

In [139]:
def column_drops(data):
    data = data.drop(['WebUrl',"CheckInDate Date","Aux1", "Aux5","FirstPrintDate Time","FirstPrintLocId", 'FirstPrintStationId','CheckInDate Time',"ModifyDate Time", "IndexDate"], axis = 1)
    data = data.drop(['RegTypeDesc','SourceApp','CountryName','StreamSteps','StreamName', 'RegType','StreamStep',"AddUserId","ModifyUserId","AddDate Time"], axis = 1)
    data = data.drop(["Pwd", "UserName", "AccountKeyPublic","AccountKeyPrivate", "Notes",'Id', 'AccountId', 'ClientIp', 'StreamId','State', 'StreamStatus'], axis = 1)
    return data

In [140]:
def data_dummies(data):
    data = pd.get_dummies(data,columns = ['Aux6', 'ApprovalUserId', 'LangId', 'MemberStatus', 'TermsAccepted'], dummy_na = True)
    return data
    

In [141]:
def data_dummies_cleanup(data):
    d = {'3001': ['3001', '3001.1']}   
    data = data.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
    d = {'2001': ['2001', '2001.1']}
    data = data.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
    # renames duplicate column names
    d = {'TermsAccepted_nan': ['TermsAccepted_nan', 'TermsAccepted_nan.1']}
    data = data.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
    d = {'MemberStatus_nan': ['MemberStatus_nan', 'MemberStatus_nan.1']}
    data = data.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
    return data

In [146]:
def processing_data(data):
    date_checker = datetime.datetime(2016,7,23) #train_X 2016,6,3 2016,7,23    test_X 2017,5,24  2017,7,13   
    data = zip_distance(data)
    data = data.apply(pd.to_numeric, errors='ignore')
    data = clear_dups_na(data)
    data = def_time_modify(data, date_checker)
    data = date_time_update(data, date_checker)                      #DONT FORGET TO CHANGE THIS UP HERE! THE DATES!!!
    data = data_drop_other_text(data)
    data = get_rid_of_nonnumerical_answer(data)
    data = answers_dummies(data)
    data = data_change_string_drop_dups(data)
    data = purchase_dummies(data)
    data = state_dummies(data)
    data = email_dummies(data)
    data = column_drops(data)
    data = data_dummies(data)
    data = data_dummies_cleanup(data)
    return data

In [144]:
test_data = processing_data(test_data)

In [145]:
test_data.Distance.value_counts()

nan              669
8311.53649198    403
238.142872527    121
8929.99114339    102
24.1011894369     85
12.6881755978     70
13.5118945605     68
619.316904052     66
22.6114343371     66
11.0882285693     64
11.2060057264     64
16.8850855957     62
2250.19822755     57
629.658024429     55
22.8625825643     54
29.3606737306     53
240.474544128     51
9539.50296971     48
630.537340659     46
223.265620559     45
16.3510640211     45
0.0               44
24.0282595168     44
196.354932766     43
616.018464698     42
217.933598322     41
19.9885974154     38
87.1691187086     38
22.4046895685     38
20.3752471783     37
                ... 
1518.81366811      1
3225.80141754      1
125.500131602      1
2006.16510934      1
1684.11413345      1
2005.39380805      1
218.298666094      1
268.77899304       1
454.600557328      1
19.5071858454      1
1068.37669541      1
876.175377633      1
341.292033843      1
1065.54946494      1
2395.85369586      1
363.60732741       1
1714.49902905

In [147]:
data = processing_data(data)

In [ ]:
data

In [3]:
data.to_pickle("Data_Zoo.pkl")

NameError: name 'data' is not defined

In [148]:
test_data.to_pickle("Data_Zoo_test.pkl")

In [ ]:
START HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [67]:
data = pd.read_pickle("Data_Zoo.pkl")

In [68]:
test_data = pd.read_pickle("Data_Zoo_test.pkl")

In [69]:
data.shape

(12875, 357)

In [70]:
test_data.shape

(10242, 349)

In [71]:
#(data.T.drop_duplicates(subset=data.index)).T  # to big of a dataset to use this to get rid of duplicate columns with different names

In [72]:
#for a in range(0,len(data.columns)):                                          # look at values of columns
#    if len(data.iloc[:,1])-data.iloc[:,a].isnull().sum() == 0:
#         print data.columns[a], len(data.iloc[:,1])-data.iloc[:,a].isnull().sum()

In [73]:
def day_to_event(data, test_data, date, test_date):
    data["Days_to_Event"] = (date - data['AddDate Date']).dt.days
    test_data["Days_to_Event"] = (test_date - test_data['AddDate Date']).dt.days
    return data, test_data

In [74]:
def day_to_event_modify(data, test_data, date, test_date):
    data["Days_to_Event_Modify"] = (date - data['ModifyDate Date']).dt.days
    test_data["Days_to_Event_Modify"] = (test_date - test_data['ModifyDate Date']).dt.days
    return data, test_data

In [75]:
def y_for_train(train_data):
    y =[]
    for a in train_data['FirstPrintDate Date']:
        if a == 'nan':
            y.append(0)
        else:
            y.append(1)
    return y

In [76]:
def y_for_test(test_data):
    y =[]
    for a in test_data['FirstPrintDate Date']:
        if a == 'nan':
            y.append(0)
        else:
            y.append(1)
    return y

In [77]:
def train_test_column_sync(data, test_data):
    col=[]
    for a in data.columns:
        for b in test_data.columns:
            if a == b:
                col.append(b)    
    return col

In [78]:
def data_impute_Distance(data):
    imp=Imputer(missing_values="NaN", strategy="mean" )
    data["Distance"]=imp.fit_transform(data[["Distance"]]).ravel()
    return data

In [79]:
def data_impute(X):
    data_X_hold = X.columns
    imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
    imp.fit(X)
    data_T=imp.transform(X)
    X = pd.DataFrame(data_T, columns = data_X_hold)
    return X

In [80]:
def train_test_processing(data, test_data):
    date = datetime.datetime(2016,7,23)
    test_date = datetime.datetime(2017,7,13)
    train_Y = y_for_train(data)
    test_Y = y_for_test(test_data)
    data = data_impute_Distance(data)
    test_data = data_impute_Distance(test_data)
    data, test_data = day_to_event(data, test_data, date, test_date)
    data, test_data = day_to_event_modify(data, test_data, date, test_date)
    test_data = test_data.apply(pd.to_numeric, errors='ignore')
    train_X = data.select_dtypes(include = ['int64', 'float64', 'uint8', 'bool']).iloc[:, 0:]
    test_X = test_data.select_dtypes(include = ['int64', 'float64', 'uint8', 'bool']).iloc[:, 0:]
    col = train_test_column_sync(train_X, test_X)
    train_X = data[col]
    test_X = test_data[col]
    train_X = data_impute(train_X)
    test_X = data_impute(test_X)
    return (train_X, train_Y, test_X, test_Y)

In [81]:
train_X, train_Y, test_X, test_Y = train_test_processing(data, test_data)

In [82]:
train_X.loc[:,'sum_response'] = train_X.loc[:,'0201':'508'].sum(axis=1)
test_X.loc[:,'sum_response'] = test_X.loc[:,'0201':'508'].sum(axis=1)

In [83]:
import pandas_ml as pdml
import imblearn

In [84]:
train_Y = pd.DataFrame(train_Y)

In [85]:
data_normalized = pd.concat([train_X, train_Y], axis = 1)


In [86]:
data_normalized = pdml.ModelFrame(data_normalized, target = 0)

In [87]:
sampler = data_normalized.imbalance.over_sampling.SMOTE()
sampled = data_normalized.fit_sample(sampler)
sampled=pd.DataFrame(sampled)

In [88]:
sampled = sampled.sample(frac=1)
train_Y = sampled[0]
train_X = sampled.drop([0], axis = 1)

In [90]:
train_X.columns

Index([u'Distance', u'Phone1CountryPrefix', u'Phone2Number',
       u'FaxCountryPrefix', u'AmountBilled', u'AmountPaid', u'AmountDue',
       u'0201', u'0202', u'0203',
       ...
       u'MemberStatus_FullMember', u'MemberStatus_NonMember',
       u'MemberStatus_nan', u'MemberStatus_nan.1', u'TermsAccepted_accepted',
       u'TermsAccepted_nan', u'TermsAccepted_nan.1', u'Days_to_Event',
       u'Days_to_Event_Modify', u'sum_response'],
      dtype='object', length=304)

In [207]:
train_T = train_X[['Days_to_Event','Days_to_Event_Modify','TermsAccepted_accepted','Email_Username','AmountBilled','PET','Distance','CA', 'NV', 'AZ','TX','WA','CO','sum_response']]

In [203]:
train_T.columns

Index([u'Days_to_Event', u'Days_to_Event_Modify', u'Email_Username', u'PET',
       u'Distance', u'CA', u'NV', u'AZ', u'TX', u'WA', u'CO', u'sum_response'],
      dtype='object')

In [208]:
num_folds = 10
num_instances = len(train_T) 
seed = 7
scoring = 'roc_auc'

In [209]:
from sklearn.svm import LinearSVR

In [210]:
models=[]
models.append(('LR', LogisticRegression(random_state = seed)))
models.append(('LASSO', Lasso())) 
models.append(('Ridge', Ridge())) 
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('NB', GaussianNB()))
models.append(('KNeighborsClassifier', KNeighborsClassifier()))#ewights = 'distance' 
models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = seed)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = seed)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state = seed)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state = seed)))
#models.append(('SVC', SVC(kernel = 'linear', random_state = seed)))
#models.append(('LinearSVR', LinearSVR(random_state = seed)))
for name, model in models:
    kfold = model_selection.KFold(shuffle = True, n_splits=num_folds, random_state=seed)
    
    cv_results = model_selection.cross_val_score(model, train_T, train_Y, cv=kfold, scoring = scoring)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.748413 (0.008328)
LASSO: 0.587483 (0.006108)
Ridge: 0.723309 (0.008809)
LDA: 0.723316 (0.008816)
NB: 0.695429 (0.008781)
KNeighborsClassifier: 0.745531 (0.007023)
XGBClassifier: 0.813534 (0.008151)
GradientBoostingClassifier: 0.817716 (0.008085)
AdaBoostClassifier: 0.776325 (0.009434)
RandomForestClassifier: 0.847637 (0.009854)
ExtraTreesClassifier: 0.835038 (0.008808)
DecisionTreeClassifier: 0.761498 (0.012823)


In [51]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [ ]:
train_P= test_X[['Days_to_Event','Days_to_Event_Modify','TermsAccepted_accepted','Email_Username','PET','Distance']]

In [211]:
model = LinearDiscriminantAnalysis()
model.fit(train_T, train_Y)
#test_preds = model.predict(test_X)
array = model.predict(train_T)
probab = model.predict_proba(train_T)


In [214]:
print metrics.accuracy_score(train_Y, array)
print metrics.roc_auc_score(train_Y, probab[:, 1])

0.647284929835
0.724450384001


In [ ]:
probab[3]

In [200]:
unique_elements, counts_elements = np.unique(array, return_counts=True)  # this is predictions
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [8201 8189]]


In [ ]:
unique_elements, counts_elements = np.unique(train_Y, return_counts=True) #this is actual
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

In [497]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [213]:
confusion_matrix(train_Y, array)

array([[4328, 3867],
       [1914, 6281]], dtype=int64)

In [156]:
estimator = LogisticRegression(random_state = seed)
rfe = RFECV(estimator,cv = kfold)
fit = rfe.fit(train_T,train_Y)
print("Num of feature: %d") % fit.n_features_
#print("Selected features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

Num of feature: 16
Feature Ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
top_65_important_features = [] 
for b in range(0,len(fit.ranking_)):
    if fit.ranking_[b] == 1:
        top_65_important_features.append(b)
        print b,train_X.columns[b]

In [ ]:
new_X = train_X.iloc[:,top_65_important_features]

In [ ]:
X = new_X

In [ ]:
top_65_important_features = X

In [123]:
models=[]
models.append(('LR', LogisticRegression(random_state = seed)))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('NB', GaussianNB()))
models.append(('KNeighborsClassifier', KNeighborsClassifier())) 
models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = seed)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = seed)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state = seed)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state = seed)))
#models.append(('SVC', SVC(kernel = 'linear', random_state = seed)))
#models.append(('LinearSVR', LinearSVR(random_state = seed)))
for name, model in models:
    kfold = model_selection.KFold(shuffle = True, n_splits=num_folds, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, train_Y, cv=kfold, scoring= scoring)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

NameError: name 'X' is not defined

In [ ]:
y_test = pd.DataFrame({'y': y})

In [ ]:
Z=[]
Z=pd.concat([X,train_Y],axis=1)

In [ ]:
corr = Z.select_dtypes(include = ['int64', 'float64','uint8']).iloc[:, 1:].corr()

In [ ]:
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(60, 60))
with sns.axes_style("white"):
    ax = sns.heatmap(corr, mask=mask, vmin = -1, vmax=1, square=True)

In [ ]:

pd.set_option('display.max_columns', 500)

In [127]:
for a in range(0, len(models)):
    model1 = models[a]
    for b in range(a+1, len(models)):
        model2 = models[b]
        for c in range(b+1, len(models)):
            model3 = models[c]
            estimators = []
            estimators.append(model1)
            estimators.append(model2)
            estimators.append(model3)
            ensemble = VotingClassifier(estimators, voting='soft')
            results = model_selection.cross_val_score(ensemble, train_T, train_Y, cv=kfold, scoring= scoring)
            if results.mean() > .85:
                print(model1[0], model2[0], model3[0],results.mean(), results.std())


('LR', 'KNeighborsClassifier', 'RandomForestClassifier', 0.85369773310560304, 0.0074974399523090171)
('LR', 'KNeighborsClassifier', 'ExtraTreesClassifier', 0.85493828459381971, 0.0087903236616036669)
('LR', 'XGBClassifier', 'RandomForestClassifier', 0.85486887655878885, 0.0084441092122449091)
('LR', 'XGBClassifier', 'ExtraTreesClassifier', 0.85615346320586616, 0.0087860251358491352)
('LR', 'GradientBoostingClassifier', 'RandomForestClassifier', 0.85548179096871357, 0.0087556884215473311)
('LR', 'GradientBoostingClassifier', 'ExtraTreesClassifier', 0.85671676466978164, 0.0090803949059594762)
('LR', 'AdaBoostClassifier', 'RandomForestClassifier', 0.85188208214529548, 0.0088626699608252077)
('LR', 'RandomForestClassifier', 'ExtraTreesClassifier', 0.85688839965691055, 0.0093673845581198571)
('LDA', 'XGBClassifier', 'ExtraTreesClassifier', 0.85002648382492585, 0.0097600352542832527)
('LDA', 'GradientBoostingClassifier', 'ExtraTreesClassifier', 0.85067654632557921, 0.0099652017836868688)
('L

In [144]:
estimators = []

estimators.append(('LR',LogisticRegression(random_state = seed)))
estimators.append(('XGBClassifier',xgb.XGBClassifier()))
estimators.append(('KNeighborsClassifier', KNeighborsClassifier()))
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(train_T, train_Y)
predict = ensemble.predict(test_P)
probab = ensemble.predict_proba(test_P)

In [139]:
results = model_selection.cross_val_score(ensemble, train_T, train_Y, cv=kfold, scoring= scoring)

In [ ]:
estimators = ('LR', LogisticRegression(random_state = seed)'LR', 'XGBClassifier', 'RandomForestClassifier', 0.85486887655878885

In [ ]:
RandomForestClassifier(random_state = seed)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))

In [215]:
test_P = test_X[['Days_to_Event','Days_to_Event_Modify','TermsAccepted_accepted','Email_Username','AmountBilled','PET','Distance','CA', 'NV', 'AZ','TX','WA','CO','sum_response']]

In [216]:
model = LogisticRegression(random_state = seed)
model.fit(train_T,train_Y)
predict = model.predict(test_P)
probab = model.predict_proba(test_P)

In [217]:
print metrics.accuracy_score(test_Y, predict)
print metrics.roc_auc_score(test_Y, probab[:, 1])

0.744288224956
0.737400592742


In [218]:
confusion_matrix(test_Y, predict)

array([[1408, 1708],
       [ 911, 6215]], dtype=int64)

In [ ]:
c = 0
e = 0
for idx, val in enumerate(probab):
    if val[0] > .80:
        c += 1
        if test_Y[idx] == 0:
            print c, 0, idx, val[0]
        else: 
            e += 1
            print c, 0, idx, val[0], "error"
            
   
    if val[1] > .80:
        c += 1
        if test_Y[idx] == 1:
            print c, 1, idx, val[1]
        else:
            e += 1
            print c, 1, idx, val[1], "error"
print e
        


In [ ]:
from sklearn import metrics

In [ ]:
estimators = []
estimators.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = seed)))
estimators.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = seed)))
estimators.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X, train_Y)
predict = ensemble.predict(test_X[X.columns])
probab = ensemble.predict_proba(test_X[X.columns])

In [ ]:
print metrics.accuracy_score(test_Y, predict)
print metrics.roc_auc_score(test_Y, probab[:, 1])

In [ ]:
print metrics.confusion_matrix(test_Y, predict)
#print metrics.classification_report(test_Y, predict)